In [ ]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [ ]:
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GENIE_ENV") && ENV["GENIE_ENV"] == "dev")
    if (haskey(ENV, "GITPOD_REPO_ROOT"))
        run(```psql -f sqlsnippets/droptables.sql```)
    else
        println("execute this in Your REPL")
        println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)""")
    end
end

Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [ ]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


Create a Partner

In [ ]:
# create Partner
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

In [ ]:
find(Partner)

Create tariffs

In [30]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
end


create_tariff("Life Risk Insurance")
create_tariff("Terminal Illness")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 14 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'14' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 14 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Se

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 15 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'15' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions"

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffrevisions"."id" AS "tariffrevisions_id", "tariffrevisions"."ref_component" AS "tariffrevisions_ref_component", "tariffrevisions"."ref_validfrom" AS "tariffrevisions_ref_validfrom", "tariffrevisions"."ref_invalidfrom" AS "tariffrevisions_ref_invalidfrom", "tariffrevisions"."description" AS "tariffrevisions_description" FROM "tariffrevisions" ORDER BY tariffrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


2-element Vector{TariffRevision}:
 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Life Risk Insurance |
| id::DbId              | 12                  |
| ref_component::DbId   | 12                  |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 14                  |

 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Terminal Illness    |
| id::DbId              | 13                  |
| ref_component::DbId   | 13                  |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 15                  |


In [29]:


map(find(Tariff)) do t
    SearchLight.delete(t)
end

find(Tariff)

#   t2 = Tariff()
#   tr2 = TariffRevision(description="Terminal Illness")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#   create_entity!(w0)
#   create_component!(t2, tr2, w0)
#   commit_workflow!(w0)
#
#   t3 = Tariff()
#   tr3 = TariffRevision(description="Occupational Disability")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#
#   create_entity!(w0)
#   create_component!(t3, tr3, w0)
#   commit_workflow!(w0)
#
#   t4 = Tariff()
#   tr4 = TariffRevision(description="Profit participation")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#   create_entity!(w0)
#   create_component!(t4, tr4, w0)
#   commit_workflow!(w0)


┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: DELETE FROM tariffs WHERE id = '10'
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: DELETE FROM tariffs WHERE id = '11'
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


Tariff[]